# The Framework

## Building Blocks
The BuildingBlock interface is the core concept of the Automation-V2 interface. It
is the main extension point used to extend the framework. It allows registering 
python code that will be executed when a line of RVT is encountered.

At it's core a BuildingBlock is very simple. It has 3 parts:

1. A function `name` for resolving the BuildingBlock name. Defaults to the classname
2. A function `check_syntax` to determine if the arguments are valid for this BuildingBlock
3. A function `execute` to run the block. Returns a `BlockResult` 

In [2]:
# export

class BuildingBlock:
    ''' 
    The 'BuildingBlock' of the automation framework. Registers an function to
    be run during text execution.
    '''
    def name(self):
        '''Returns the name of the building block. The name is used
        as a first order lookup for the block'''
        return type(self).__name__
    
    def check_syntax(self, *args):
        '''Returns True if this BuildingBlock can support the arguments and False otherwise'''
        return True
    
    def execute(self, *args):
        '''Executes the block. Returns a BlockResult'''
        return BlockResult(False)
    

In [3]:
# export

class BlockResult(object):
    '''
    The result of executing a BuildingBlock
    '''
    def __init__(self, passed, stdout="", stderr=""):
        self.passed = passed
        self.stdout = stdout
        self.stderr = stderr
        
    def __bool__(self):
        return self.passed
    
    def __str__(self):
        return "<BlockResult: %s, %s, %s>" % ('PASS' if self.passed else 'FAIL', self.stdout, self.stderr)
    

In [4]:
# Unit tests

class TestBlock(BuildingBlock):
    pass

class TestBlock2(BuildingBlock):
    def name(self): return 'CustomName'
    
    
assert TestBlock().name() == 'TestBlock'
assert TestBlock2().name() == 'CustomName'

## Observers
Executing a test simply results in None if all tests have passed or a failed block result. 
We are going to want more detail as test executes and the `Observer` interface will give us that.
Observers will register themselves with an `ObserverManager` and get notified when key events
happen during test execution. One thing to note here is that we want observers to be open for extenson
and so we will use a convention that any method that begins with an 'on_' will be a observer event. Because
of this there is no specific Observer class as any python class with do.

This means an observer can implement any number of events.

In [5]:
#export

from functools import partial
  
class Observer:
    pass

class ObserverManager:
    def __init__(self):
        self.observers = set()
    
    def add_observer(self, observer):
        self.observers.add(observer)
        
    def notify(self, event, *args, **kwargs):
        for observer in self.observers:
            if hasattr(observer, 'on_' + event):
                getattr(observer, 'on_' + event)(observer, *args, **kwargs)
            
    def __getattr__(self, name):
        if name.startswith('on_'):
            return partial(self.notify, name[3:])
        else:
            raise AttributeError
    
    # These are provided to document the usual events.
    # Observers are NOT limited to just these events
    
    def on_procedure_begin(self, *args, **kwargs):             self.notify('procedure_begin', *args, **kwargs)
    def on_step_start(self, *args, **kwargs):                  self.notify('step_start', *args, **kwargs)
    def on_step_end(self, *args, **kwargs):                    self.notify('step_end', *args, **kwargs)
    def on_procedure_end(self, *args, **kwargs):               self.notify('procedure_end', *args, **kwargs)
    def on_comment(self, *args, **kwargs):                     self.notify('comment', *args, **kwargs)

        
# TODO: Decide what is sent in each observer. Maybe remove the need to have *args and just send in a dict

In [6]:
# Unit Tests
manager = ObserverManager()

class TestObserver(object):
    def __init__(self):
        self.event_history = []
        
    def on_procedure_begin(self, *args, **kwargs):
        self.event_history.append(('procedure_begin', args[1:], kwargs))

    def on_custom_event(self, *args, **kwargs):
        self.event_history.append(('custom_event', args[1:], kwargs))
        
args1,   args2   = (1, 2, 3),        ('a', 'b', 'c')
kwargs1, kwargs2 = {'a': 1, 'b': 2}, {'1': 'a', '2': 'b'}

# Setup
observer = TestObserver()
manager.add_observer(observer)

# Execute
manager.on_procedure_begin(*args1, **kwargs1)
manager.on_comment()
manager.on_custom_event(*args2, **kwargs2)

# Assert
assert observer.event_history == [('procedure_begin', args1, kwargs1), ('custom_event', args2, kwargs2)]

In [8]:
# Lets create a jupyter aware observer

from IPython.display import display, HTML
import random

def display_in_div(id, innerHTML, append=True):
    append_or_set = '+=' if append else '='
    display(HTML(f'''
        <script id="to-remove">
            (function() {{
                var element = document.getElementById("rand-{id}")
                const innerHTML = `{innerHTML}`
                element.innerHTML {append_or_set} innerHTML
            
                // Remove ourself and the jupyter generated divs from the DOM
                var to_remove = document.getElementById('to-remove').parentNode.parentNode;
                to_remove.parentNode.removeChild(to_remove) 
            }})()
        </script>
        '''))
    
class JupyterObserver:
    def __init__(self):
        self.current_step = 0
        self.current_procedure = 0
        
    def on_procedure_begin(self, *args, **kwargs):
        self.current_procedure = random.randint(0, 4000000)
        
        display(HTML(f'''
        <div id="rand-{self.current_procedure}" style="border: solid; border-width: 1px; border-radius: 5px; padding: 5px">
            <h2>{kwargs["name"]}</h2>
        </div>'''))
        
    def on_comment(self, *args, **kwargs):
        display_in_div(id=self.current_procedure, 
                       innerHTML=f'<div><span style="color: grey">{kwargs["line"]}</span></div>')
        
    def on_step_start(self, *args, **kwargs):
        self.current_step = random.randint(0, 4000000)
        display_in_div(id=self.current_procedure,
                       innerHTML=f'''
        <div>
            <pre style="display: inline-block; background: inherit">{kwargs["line"]}</pre>
            <div id="rand-{self.current_step}" style="float:right; display: inline-block">
        </div>''')
        
    def on_step_end(self, *args, **kwargs):
        result = kwargs['result']
        result_str, color = ('PASSED', 'green') if result else ('FAILED', 'red')
        display_in_div(id=self.current_step,
                       innerHTML=f'<span style="float:right; color: {color}">{result_str}</span>')
            
    def on_procedure_end(self, *args, **kwargs):
        result = kwargs['result']
        result_str, color = ('PASSED', 'green') if result else ('FAILED', 'red')
        display_in_div(id=self.current_procedure,
                       innerHTML=f'<h3>TEST <span style="color: {color}">{result_str}</span></h3>')
    

In [9]:

# Lets test this
manager = ObserverManager()
manager.add_observer(JupyterObserver())

# Execute
manager.on_procedure_begin(name="Procedure1")
manager.on_comment(line="# Some comment about nothing")
manager.on_step_start(line="Wait 10")
time.sleep(10)
manager.on_step_end(result=BlockResult(True))
manager.on_procedure_end(result=BlockResult(True))

NameError: name 'time' is not defined